In [3]:
import cPickle as pickle
from nltk.stem import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet as wn
import json


In [4]:
wnl = WordNetLemmatizer()

In [5]:
with open('./data/imagenet_top1k_captions.pkl', 'rb') as f:
    captions = pickle.load(f)
with open('./data/top1k.pkl', 'rb') as f:
    top1k = dict(pickle.load(f))
    


In [18]:
errPerWord = {}
for k,v in captions.iteritems():
    w = top1k[k]
    wl = wnl.lemmatize(w)
    print w, wl
    errors = sum([1 for cap in v if wl not in map(wnl.lemmatize, nltk.word_tokenize(cap))])
    print errors, len(v), errors*100.0/len(v)

woman woman
523 1405 37.2241992883
street street
706 1297 54.433307633
person person
1209 1242 97.3429951691
people people
462 1431 32.285115304
white white
963 1243 77.47385358


In [7]:
captions.keys()

[u'n10787470', u'n04334599', u'n00007846', u'n07942152', u'n09638875']

In [8]:
for k in captions:
    print top1k[k]

woman
street
person
people
white


# Flexible Matching
- Direct hyponyms
- full hyponyms
- Vector similarity 

In [31]:
captions[u'n09638875']

[u'a woman with a white shirt and black tie .',
 u'a woman wearing a tie and sunglasses wearing a suit .',
 u'a woman in a black shirt and a black tie .',
 u'a man and a woman are eating a piece of food .',
 u'a man with a beard and a black tie .',
 u'a young girl is sitting on a bench with her cell phone .',
 u'a group of people standing around a stone structure with a large clock .',
 u'a man is sitting on a bench reading a book .',
 u'a woman sitting on a bench with a hat on her head .',
 u'a man with glasses is looking at his cell phone .',
 u'a man in a red shirt is holding a white umbrella .',
 u'a woman with a white shirt and a white dog .',
 u'a woman wearing a tie and a white shirt .',
 u'a man wearing a suit and tie with a beard in his mouth .',
 u'a woman is holding a glass of wine .',
 u'a young boy standing on a bench with a skateboard .',
 u'a man brushing his teeth with a white toothbrush .',
 u'a woman is holding a dog in her arms .',
 u'a young girl is wearing a white 

In [22]:
# Direct hyponyms
person = wn.synsets('person')[0]

def direct_hyponyms(synset): 
    direct_hypo = [] 
    for ss in synset.hyponyms():
        direct_hypo += ss.lemma_names()
    direct_hypo =  sorted(direct_hypo) 
    return direct_hypo


In [38]:
# Full hyponyms
def full_hyponyms(ss):
    full_hypo = []
    q = [ss]
    while len(q) > 0:
        a = q.pop(0)
        q += a.hyponyms()
        full_hypo += [l.replace('_',' ') for l in a.lemma_names()]
    return full_hypo

full_hypo = full_hyponyms(person)
print len(full_hypo)
print full_hypo.index('person')
# print sorted(full_hypo)
print full_hypo[:5]

13235
0
[u'person', u'individual', u'someone', u'somebody', u'mortal']


In [33]:
# Get Direct hypo
from collections import Counter
errPerWord = {}
for k,v in captions.iteritems():
    w = top1k[k]
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss
    direct_hypo = direct_hyponyms(ss)
    counter = Counter()
    countErr = 0
    
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in direct_hypo:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)

Synset('woman.n.01')
80 1405 5.69395017794
Synset('street.n.01')
19 1297 1.46491904395
Synset('person.n.01')
906 1242 72.9468599034
Synset('people.n.01')
3 1431 0.20964360587
Synset('white.n.01')
0 1243 0.0


In [36]:
# Get full hypo
from collections import Counter
errPerWord = {}
for k,v in captions.iteritems():
    w = top1k[k]
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss
    full_hypo = full_hyponyms(ss)
    counter = Counter()
    countErr = 0
    
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in full_hypo or w == wl:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)
    print counter

Synset('woman.n.01')
933 1405 66.4056939502
Counter({u'woman': 902, u'cat': 52, u'girl': 29, u'bird': 14, u'baby': 4})
Synset('street.n.01')
609 1297 46.9545104086
Counter({u'street': 695, u'alley': 19})
Synset('person.n.01')
1201 1242 96.6988727858
Counter({u'man': 901, u'woman': 238, u'suit': 124, u'beard': 117, u'red': 90, u'dog': 75, u'front': 72, u'hand': 48, u'cat': 43, u'person': 33, u'boy': 32, u'head': 30, u'chair': 15, u'mouth': 15, u'bird': 13, u'bag': 12, u'bear': 11, u'girl': 9, u'child': 8, u'trick': 8, u'back': 7, u'computer': 7, u'controller': 7, u'face': 6, u'pink': 6, u'grass': 6, u'rock': 5, u'fence': 4, u'baby': 4, u'cow': 4, u'brick': 3, u'sheep': 2, u'image': 1, u'competition': 1, u'monitor': 1, u'sun': 1, u'skateboarder': 1, u'power': 1, u'cowboy': 1, u'counter': 1})
Synset('people.n.01')
1286 1431 89.8672257163
Counter({u'people': 1046, u'woman': 331, u'young': 33, u'living': 3, u'few': 1})
Synset('white.n.01')
280 1243 22.52614642
Counter({u'white': 366})


In [ ]:
# Full hyponyms with Depth
def full_hyponyms_with_depth(ss):
    full_hypo = {}
    q = [{'synset':ss, 'depth':0}]
    while len(q) > 0:
        a = q.pop(0)
        q += [{'synset': s, 'depth': a['depth']-1} 
             for s in a.hyponyms()]
        for name in a.lemma_names():
            full_hypo[name] = a['depth']
    return full_hypo

In [ ]:
# Get full hypo with Depth
from collections import Counter
errPerWord = {}
for k,v in captions.iteritems():
    w = top1k[k]
    wl = wnl.lemmatize(w)
    ss = wn._synset_from_pos_and_offset('n',int(k[1:]))
    print ss
    full_hypo = full_hyponyms_with_depth(ss)
    counter = Counter()
    countErr = 0
    
    for cap in v:
        cap_lema = [wnl.lemmatize(w) for w in nltk.word_tokenize(cap)]
        isContains = False
        for w in cap_lema:
            if w in full_hypo or w == wl:
                counter[w] += 1
                isContains = True
        if isContains:
            countErr += 1
            
    print countErr, len(v), countErr*100.0/len(v)
    print counter

In [ ]:
start = datetime.now()
    
caption_file = 'data/annotations/captions_train2014.json'
image_dir = 'image/train2014_resized'
max_length = 15
word_count_threshold = 1
vgg_model_path = './data/imagenet-vgg-verydeep-19.mat'
batch_size = 50

print '1. Building Top 1K dictionary from Train dataset'

if not os.path.exists('./data/top1k.pkl'):
    train_dataset = _process_caption_data(caption_file=caption_file,
                                      image_dir=image_dir,
                                      max_length=max_length)
    word_to_idx = _build_vocab(annotations=train_dataset, threshold=word_count_threshold)
    save_pickle(word_to_idx, './data/word_to_idx.pkl')
    top1k = _build_top1k_vocab(train_dataset)
    save_pickle(top1k, './data/top1k.pkl')
else:
    top1k = load_pickle('./data/top1k.pkl')

In [40]:
ss = wn._synset_from_pos_and_offset('n',9638875)

In [41]:
ss.lemma_names()


[u'White', u'White_person', u'Caucasian']

# Let's build Top 1k using Part of Speech Tagger 

In [65]:
caption_file = 'data/annotations/captions_train2014.json'
from nltk.tag.stanford import StanfordPOSTagger
from collections import Counter
import json
def get_top1k_noun(caption_file):
    counter = Counter()
    with open(caption_file) as f:
        caption_data = json.load(f)
    st = StanfordPOSTagger('stanford-postagger-2016-10-31/models/english-left3words-distsim.tagger',
                           path_to_jar='stanford-postagger-2016-10-31/stanford-postagger.jar')
    
    for anno in caption_data['annotations'][:300]:
        counter += Counter(st.tag(anno['caption'].split()))
        
    nouns = [c for c in counter.most_common(3000) if c[0][1] == 'NN']
    
    return nouns[:1000]
        

In [67]:
%%timeit print get_top1k_noun(caption_file)

UsageError: %%timeit is a cell magic, but the cell body is empty. Did you mean the line magic %timeit (single %)?

In [60]:
import ipdb